# NLP

# LAB 2 - Lemmatization and full text search

In [66]:
import os
import numpy as np
from elasticsearch import Elasticsearch

In [67]:
es = Elasticsearch()

In [68]:
es.info()['version']

{'number': '7.9.2',
 'build_flavor': 'default',
 'build_type': 'zip',
 'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
 'build_date': '2020-09-23T00:45:33.626720Z',
 'build_snapshot': False,
 'lucene_version': '8.6.2',
 'minimum_wire_compatibility_version': '6.8.0',
 'minimum_index_compatibility_version': '6.0.0-beta1'}

### Zadanie 3 i 4
Poniższe wywołanie tworzy indeks wraz z analyzerem zdefiniowanym zgodnie z treścią zadania. Dodatkowo, analyzer został ustawiony jako domyślny dla pola content w tym indeksie, aby w kolejnych zadaniach korzystać z niego bez konieczności dodawania informacji, że należy użyć właśnie jego.

In [69]:
es.indices.create(
    index="legislation_acts",
    body={
        "settings": {
            "analysis": {
                "filter": {
                  "kodeks_synonyms": {
                      "type": "synonym",
                      "synonyms": [ 
                          "kpk => kodeks postępowania karnego",
                          "kpc => kodeks postępowania cywilnego",
                          "kk => kodeks karny",
                          "kc => kodeks cywilny"]
                    }    
                },
                "analyzer": {
                  "polish_legislation_acts_analyzer": {
                      "type": "custom",
                      "tokenizer": "standard",
                      "filter": ["kodeks_synonyms", "morfologik_stem", "lowercase"]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "content": {
                    "type": "text",
                    "analyzer": "polish_legislation_acts_analyzer"
                }
            }
        }
    }
)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'legislation_acts'}

### Zadanie 5

In [70]:
path = './ustawy'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding='utf-8') as file:
        es.create("legislation_acts", filename, {"content": file.read()})

In [71]:
es.count(index="legislation_acts")['count']

1180

W indeksie znalazło się 1180 ustaw, a więc wszystkie znajdujące się w zestawie danych.

### Zadanie 6

In [72]:
es.count(
    index="legislation_acts",
    body={
        "query": {
            "match": {
                "content": {
                    "query": "ustawa"
                }
            }
        }
    }
)['count']

1179

Słowo **ustawa** zawiera 1179 z 1180 analizowanych plików.

### Zadanie 7

In [73]:
es.count(
    index="legislation_acts",
    body={
        "query": {
            "match_phrase": {
                "content": {
                    "query": "kodeks postępowania cywilnego"
                }
            }
        }
    }
)['count']

100

Wyrażenie **kodeks postępowania cywilnego** w dowolnej formie jest zawarte w 100 ustawach ze zbioru.

### Zadanie 8

In [74]:
es.count(
    index="legislation_acts",
    body={
        "query": {
            "match_phrase": {
                "content": {
                    "query": "wchodzi w życie",
                    "slop": 2
                }                             
            }
        },
    }
)['count']

1175

Frazę **wchodzi w życie** w dowolnej formie z możliwością wystąpienia do dwóch dodatkowych słów w tej frazie zawiera 1175 dokumentów.

### Zadanie 9 i 10

Poniższe zapytanie znajduje 10 dokumentów najbardziej pasujących do słowa **konstytucja**, a w każdym z nich znajduje do trzech fragmentów zawierających to słowo i sortuje wyniki na podstawie uzyskanej wartości *score*.

In [82]:
hits = es.search(
    index="legislation_acts",
    size = 10,
    body={
        "query": {
            "match": {
                "content": {
                    "query": "konstytucja",
                }                             
            }
        },
        "highlight" : {
            "fields" : {
                "content" : {}
            },
            "type":"unified",
            "boundary_scanner": "sentence",
            "number_of_fragments": 3,
            "order": "score"
        }
    }
)['hits']['hits']

Poniżej wypisane zostały nazwy plików zawierających 10 dokumentów najbardziej pasujących do słowa **konstytucja**.

In [83]:
for hit in hits:
    print('Filename: {}, score: {}'.format(hit['_id'], hit['_score']))

Filename: 1997_629.txt, score: 6.8713923
Filename: 2000_443.txt, score: 6.666975
Filename: 1997_604.txt, score: 6.6366324
Filename: 1996_350.txt, score: 6.6311083
Filename: 1997_642.txt, score: 6.2556105
Filename: 2001_23.txt, score: 6.0630813
Filename: 1996_199.txt, score: 5.9332457
Filename: 1999_688.txt, score: 5.854491
Filename: 1997_681.txt, score: 5.4713526
Filename: 2001_1082.txt, score: 5.4713526


Fragmenty ze znalezionych dokumentów zawierające słowo **konstytucja** w dowolnej formie:

In [87]:
highlights = [hit['highlight']['content'] for hit in hits]
for num, highlight in enumerate(highlights):
    print(num+1)
    for h in highlight:
        print(h)
        print()
    print(100*'-')

1
W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>

o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej

Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie

----------------------------------------------------------------------------------------------------
2
umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>

co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>

międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>

----------------------------------------------------------------------------------------------------
3
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   polityc